In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [4]:
from sklearn.preprocessing import StandardScaler

LOAD DATA

In [27]:
dataset = pd.read_csv('../../Data/train.csv',usecols=[0,4,8,9,10,11,13,14])

Use restricted dataset here ( for the last submission) only lines with context_type in [1,5,20,23], because test set only contains these 4 context_type

In [29]:
dataset = dataset[dataset['context_type'].isin([1,5,20,23])]

In [31]:
dataset.shape

(1815738, 8)

OBSOLETE

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(dataset.iloc[:,:14],dataset['is_listened'],test_size=0.2)

In [ ]:
historical_meantarget_genre = pd.concat([xtrain,ytrain],axis=1).groupby(['user_id','genre_id'],as_index=False)['is_listened'].mean()

In [ ]:
hist_nb_genre = pd.concat([xtrain,ytrain],axis=1).groupby(['user_id','genre_id'],as_index=False)['is_listened'].count()

In [ ]:
historical_meantarget_genre.columns = ['user_id','genre_id','mean_islistened']
hist_nb_genre.columns = ['user_id','genre_id','count_islistened']

In [ ]:
xtrain = xtrain.merge(historical_meantarget_genre,how='left',on=['user_id','genre_id'])
xtest = xtest.merge(historical_meantarget_genre,how='left',on=['user_id','genre_id'])

xtrain = xtrain.merge(hist_nb_genre,how='left',on=['user_id','genre_id'])
xtest = xtest.merge(hist_nb_genre,how='left',on=['user_id','genre_id'])

In [ ]:
xtest.fillna(0,inplace=True)

In [6]:
features = ['user_age','context_type','genre_id','listen_type',
            'media_duration','user_gender','release_date']

FIN OBSOLETE

#### Full data model

Ici on a d'abord chargé toute la donnée pour calculer une moyenne et un count de is_listened pour chaque user et chaque genre, ces features font overfitté à max mais ça me paraît logique de considérer les actions antérieures de chaque utilisateur face à un genre particulier

In [6]:
hist_mean_islistened = dataset.groupby(['user_id','genre_id'],as_index=False)['is_listened'].mean()

In [7]:
hist_nb_islistened = dataset.groupby(['user_id','genre_id'],as_index=False)['is_listened'].count()

In [6]:
hist_mean_islistened.columns = ['user_id','genre_id','mean_islistened']
hist_nb_islistened.columns = ['user_id','genre_id','count_islistened']

In [8]:
hist_mean_islistened.to_csv('hist_mean.csv',sep=',',index=False)
hist_nb_islistened.to_csv('hist_nb.csv',sep=',',index=False)

Load hist_mean and hist_nb computed on the entire dataset

In [8]:
hist_mean_islistened = pd.read_csv('hist_mean.csv',sep=',')
hist_nb_islistened = pd.read_csv('hist_nb.csv',sep=',')

In [32]:
dataset = dataset.merge(hist_mean_islistened,how='left',on=['user_id','genre_id'])
dataset = dataset.merge(hist_nb_islistened,how='left',on=['user_id','genre_id'])

In [33]:
dataset.shape

(1815738, 10)

release_year n'apportait au final pas de gain 

In [14]:
def extract_year(x):
    return int(str(x)[:4])

In [17]:
dataset['release_year'] = dataset['release_date'].apply(lambda x : extract_year(x))

In [18]:
dataset.describe()

,user_age,context_type,genre_id,listen_type,media_duration,user_gender,release_date,user_id,is_listened,mean_islistened,count_islistened,release_year
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1.000000e+06,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,24.008044,2.262265,1942.898121,0.282579,232.683345,0.384134,2.011225e+07,4217.182774,0.692703,0.692703,74.317344,2011.148507
std,3.822657,4.473217,13096.110213,0.450254,66.248591,0.486390,8.858665e+04,4068.074329,0.461374,0.281999,118.162162,8.859443
min,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.912121e+07,0.000000,0.000000,0.000000,1.000000,1912.000000
25%,21.000000,0.000000,0.000000,0.000000,196.000000,0.000000,2.010010e+07,952.000000,0.000000,0.516129,9.000000,2010.000000
50%,24.000000,1.000000,0.000000,0.000000,226.000000,0.000000,2.015062e+07,2899.000000,1.000000,0.764706,31.000000,2015.000000
75%,27.000000,2.000000,25.000000,1.000000,256.000000,1.000000,2.016082e+07,6376.000000,1.000000,0.925926,89.000000,2016.000000
max,30.000000,71.000000,255191.000000,1.000000,8216.000000,1.000000,2.017031e+07,19916.000000,1.000000,1.000000,1145.000000,2017.000000


# Concat data

In [34]:
listen = pd.get_dummies(dataset['listen_type'],prefix='listentype')
user = pd.get_dummies(dataset['user_gender'],prefix='sex')
context = pd.get_dummies(dataset['context_type'],prefix='context')

In [35]:
dataset = pd.concat([dataset,listen,user,context],axis=1)

In [36]:
dataset.shape

(1815738, 18)

OBSOLETE

In [37]:
dataset.columns

Index([u'genre_id', u'context_type', u'media_duration', u'listen_type',
       u'user_gender', u'user_id', u'user_age', u'is_listened',
       u'mean_islistened', u'count_islistened', u'listentype_0',
       u'listentype_1', u'sex_0', u'sex_1', u'context_1', u'context_5',
       u'context_20', u'context_23'],
      dtype='object')

In [75]:
res = []
for i in range(5):
    xtrain,xtest,ytrain,ytest = train_test_split(dataset[features],dataset['is_listened'],test_size=0.3)
    logit = LogisticRegression()
    logit.fit(xtrain,ytrain)
    res.append(roc_auc_score(ytest,logit.predict_proba(xtest[features])[:,1]))

In [76]:
res

[0.62868473384245949,
 0.62766125616011292,
 0.62941162817366347,
 0.62982110942170411,
 0.62887378081645884]

FIN OBSOLETE

Logistic

Deux modèles, 
* le premier pour les cas où on dispose d'un historique mean et count is_listened pour l'utilisateur et le genre du test set (features_with)
* le deuxième pour les cas où on a pas d'histo

In [69]:
scaler_with = StandardScaler(copy=True)
scaler_without = StandardScaler(copy=True)

In [38]:
features_with = ['user_age','media_duration','mean_islistened','count_islistened']  + dataset.columns.tolist()[10:]
features_without = ['user_age','media_duration']  + dataset.columns.tolist()[10:]

In [70]:
logit_fin_with = LogisticRegression()
logit_fin_with.fit(scaler_with.fit_transform(dataset[features_with]),dataset['is_listened'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [71]:
logit_fin_without = LogisticRegression()
logit_fin_without.fit(scaler_without.fit_transform(dataset[features_without]),dataset['is_listened'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Gradient Boosting

Prend énormément de temps si dataset gros 

In [ ]:
gb = GradientBoostingClassifier(n_estimators=100,max_depth=6)
gb.fit(scaler.fit_transform(dataset[features]),dataset['is_listened'])

In [ ]:
roc_auc_score(dataset['is_listened'],gb.predict_proba(scaler.transform(dataset[features]))[:,1])

# Prediction for test data


In [19]:
sub_data = pd.read_csv('../Data/test.csv')

In [20]:
sub_data = sub_data.merge(hist_mean_islistened,how='left',on=['user_id','genre_id'])
sub_data = sub_data.merge(hist_nb_islistened,how='left',on=['user_id','genre_id'])


In [41]:
listen_sub = pd.get_dummies(sub_data['listen_type'],prefix='listentype')
user_sub = pd.get_dummies(sub_data['user_gender'],prefix='sex')
context_sub = pd.get_dummies(sub_data['context_type'],prefix='context')
sub_data = pd.concat([sub_data,listen_sub,user_sub,context_sub],axis=1)

Split into two parts depending on whether field mean and count_islistened are NA or not

In [57]:
sub_data_with = sub_data[pd.isnull(sub_data).any(axis=1)==False].reset_index(drop=True)
sub_data_without = sub_data[pd.isnull(sub_data).any(axis=1)].reset_index(drop=True)

In [58]:
sub_data_with.head()

,sample_id,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,...,mean_islistened,count_islistened,listentype_0,listentype_1,sex_0,sex_1,context_1,context_5,context_20,context_23
0,1,2744,1479317140,876497,99692,1,19851231,0,0,307,...,0.875000,8.0,0,1,1,0,1,0,0,0
1,2,2744,1479546361,876497,99692,1,19851231,0,0,307,...,0.000000,1.0,0,1,1,0,1,0,0,0
2,3,2744,1478457729,876500,99692,1,19851231,2,1,265,...,0.666667,6.0,0,1,1,0,1,0,0,0
3,4,2744,1480448560,876504,99692,1,19851231,2,1,356,...,1.000000,28.0,0,1,1,0,1,0,0,0
4,5,2744,1479410530,876497,99692,1,19851231,0,0,307,...,0.125000,16.0,0,1,1,0,1,0,0,0


In [73]:
pred_with = pd.concat([sub_data_with['sample_id'],
                      pd.DataFrame(logit_fin_with.predict_proba(scaler_with.transform(sub_data_with[features_with]))[:,1])]
                      ,axis=1)
pred_without = pd.concat([sub_data_without['sample_id'],
                        pd.DataFrame(logit_fin_without.predict_proba(scaler_without.transform(sub_data_without[features_without]))[:,1])]
                         ,axis=1)

In [74]:
submission = pd.concat([pred_with,pred_without],axis=0)
submission.columns = ['sample_id','is_listened']

In [75]:
submission.sort(['sample_id'],inplace=True)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [76]:
submission.to_csv('5_twomodels_scaled.csv',index=False,sep=',')